In [ ]:
pip install konlpy

In [ ]:
from konlpy.tag import *
komoran = Komoran()

In [ ]:
!pip install transformers boto3

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import torch
import pandas as pd

from transformers import BertConfig , BertForSequenceClassification
from transformers import BertTokenizer

model = BertForSequenceClassification.from_pretrained('beomi/kcbert-base')
tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-base')

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

In [ ]:
def split_token_get(text,tokenizer,MAX):
  split_tokens = []
  for token in komoran.morphs(text):
    for sub_token in tokenizer.tokenize(token):
      split_tokens.append(sub_token)
  if len(split_tokens) > MAX-2:
    split_tokens = split_tokens[:(MAX-2)]
  split_tokens = ["[CLS]"]+ split_tokens + ["[SEP]"]
  return split_tokens
def get_input_ids(token , tokenizer , MAX):
  segment_ids = [0] * len(token)
  input_ids = tokenizer.convert_tokens_to_ids(token)

  input_mask = [1] * (len(input_ids) -1 )
  input_mask += [0]
  
  padding = [0] * (MAX -len(input_ids) )

  input_ids += padding
  input_mask += padding
  segment_ids += padding

  assert len(input_ids) == MAX
  return input_ids 
def get_segment_ids(token, tokenizer, MAX):
  segment_ids = [0] * len(token)
  input_ids = tokenizer.convert_tokens_to_ids(token)
  padding = [0] * (MAX -len(input_ids) )
  
  segment_ids += padding
  assert len(segment_ids) == MAX
  return segment_ids 
def get_input_mask(token, tokenizer, MAX):
  
  input_ids = tokenizer.convert_tokens_to_ids(token)

  input_mask = [1]* (len(input_ids) -1 )
  input_mask += [0]

  padding = [0] * (MAX -len(input_ids) )

  input_mask += padding

  assert len(input_mask) == MAX
  return input_mask

In [ ]:
data = pd.read_excel('실력긍부정데이터수집.xlsx')
data = data.sample(frac=1)

sentences = data['sentence']
labels = data['lable'].values


sentences1 = sentences[299:]
labels1 = labels[299:]

sentences = sentences[:299]
labels = labels[:299]

In [ ]:
data2 = pd.read_csv("ability_sentences.csv")

In [ ]:
sentences1 = data['sentence']
tokenized_texts = [split_token_get(ss, tokenizer,128) for ss in sentences1]
input_ids = [get_input_ids(ss,tokenizer,128)for ss in tokenized_texts]
segment_ids = [get_segment_ids(ss,tokenizer,128) for ss in tokenized_texts]
input_mask = [get_input_mask(ss,tokenizer,128) for ss in tokenized_texts]

input_ids = torch.tensor(input_ids)
segment_ids = torch.tensor(segment_ids)
input_mask = torch.tensor(input_mask)
labels = torch.tensor(labels1)

train_data1 = TensorDataset(input_ids,input_mask)
train_dataloader1 = DataLoader(train_data1, batch_size = 1)

In [ ]:
tokenized_texts = [split_token_get(ss, tokenizer,128) for ss in sentences]
input_ids = [get_input_ids(ss,tokenizer,128)for ss in tokenized_texts]
segment_ids = [get_segment_ids(ss,tokenizer,128) for ss in tokenized_texts]
input_mask = [get_input_mask(ss,tokenizer,128) for ss in tokenized_texts]

input_ids = torch.tensor(input_ids)
segment_ids = torch.tensor(segment_ids)
input_mask = torch.tensor(input_mask)
labels = torch.tensor(labels)

train_data = TensorDataset(input_ids,input_mask,labels)
train_dataloader = DataLoader(train_data, batch_size = 8)


tokenized_texts = [split_token_get(ss, tokenizer,128) for ss in sentences1]
input_ids = [get_input_ids(ss,tokenizer,128)for ss in tokenized_texts]
segment_ids = [get_segment_ids(ss,tokenizer,128) for ss in tokenized_texts]
input_mask = [get_input_mask(ss,tokenizer,128) for ss in tokenized_texts]

input_ids = torch.tensor(input_ids)
segment_ids = torch.tensor(segment_ids)
input_mask = torch.tensor(input_mask)
labels = torch.tensor(labels1)

train_data1 = TensorDataset(input_ids,input_mask,labels)
train_dataloader1 = DataLoader(train_data1, batch_size = 8)

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()


if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')
model.cuda()

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
import numpy as np
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import time
import datetime
import random
import numpy as np
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8)
epochs = 40
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()
eval_accuracy = 0
nb_eval_steps = 0
for i in range(0, epochs):
  optimizer.zero_grad()
  t0 = time.time()
  total_loss = 0
  model.train()
  eval_accuracy = 0
  nb_eval_steps = 0
  for step,batch in enumerate(train_dataloader):
    model.train()
    if step%100 == 0 and not step ==0:
      elapsed = format_time(time.time() - t0)
      print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed) )
    
    batch = tuple(t.to(device) for t in batch)
    input_ids , input_mask , labels = batch

    outputs = model(input_ids= input_ids,
                    token_type_ids = None,
                    attention_mask=None,
                    labels = labels)
    loss = outputs

    total_loss += loss[0].item()

    loss[0].backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
  
    optimizer.step()
    scheduler.step()

    model.zero_grad()
  avg_train_loss = total_loss / len(train_dataloader)            

  print("")
  print("  Average training loss: {0:.2f}".format(avg_train_loss))
  print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
  
  t0 = time.time()
  model.eval()
  
  eval_accuracy = 0
  nb_eval_steps = 0
  for step,batch in enumerate(train_dataloader1):
    batch = tuple(t.to(device) for t in batch)

    input_ids , input_mask, labels = batch
    outputs = model(input_ids = input_ids ,
                    token_type_ids = None , 
                    attention_mask = input_mask 
                    )
    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    labels = labels.to('cpu').numpy()
    tmp_eval_accuracy = flat_accuracy(logits, labels)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1
  print("예측 Accuracy: {0:.5f}".format(eval_accuracy/nb_eval_steps))
  print("  Validation took: {:}".format(format_time(time.time() - t0)))


  Average training loss: 0.64
  Training epcoh took: 0:00:09
욕설 예측도 Accuracy: 0.65385
  Validation took: 0:00:01

  Average training loss: 0.39
  Training epcoh took: 0:00:09
욕설 예측도 Accuracy: 0.76923
  Validation took: 0:00:01

  Average training loss: 0.23
  Training epcoh took: 0:00:09
욕설 예측도 Accuracy: 0.75000
  Validation took: 0:00:01

  Average training loss: 0.09
  Training epcoh took: 0:00:09
욕설 예측도 Accuracy: 0.81731
  Validation took: 0:00:01

  Average training loss: 0.11
  Training epcoh took: 0:00:08
욕설 예측도 Accuracy: 0.75962
  Validation took: 0:00:01

  Average training loss: 0.08
  Training epcoh took: 0:00:08
욕설 예측도 Accuracy: 0.77885
  Validation took: 0:00:01

  Average training loss: 0.02
  Training epcoh took: 0:00:08
욕설 예측도 Accuracy: 0.77885
  Validation took: 0:00:01

  Average training loss: 0.08
  Training epcoh took: 0:00:08
욕설 예측도 Accuracy: 0.77885
  Validation took: 0:00:01

  Average training loss: 0.00
  Training epcoh took: 0:00:08
욕설 예측도 Accuracy: 0.81731
 

In [ ]:
torch.save(model.state_dict(), '/content/drive/My Drive/recent_model.pt')

In [ ]:
def softmax(a):
  c = np.max(a)
  exp_a = np.exp(a-c)
  sum_exp_a = np.sum(exp_a)
  y = exp_a / sum_exp_a
  return y

In [ ]:
model.eval()
for step,batch in enumerate(train_dataloader1):
    batch = tuple(t.to(device) for t in batch)

    input_ids , input_mask = batch
    outputs = model(input_ids = input_ids ,
                    token_type_ids = None , 
                    attention_mask = input_mask 
                    )
    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    print(softmax(logits)[0])
    data2['label'].loc[step] = round(softmax(logits)[0][1],4)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1
  

[9.9995494e-01 4.5102839e-05]
[5.510646e-05 9.999449e-01]
[9.9997449e-01 2.5532549e-05]
[1.8832194e-05 9.9998116e-01]
[2.1312795e-05 9.9997866e-01]
[2.5699588e-05 9.9997425e-01]
[1.8389519e-05 9.9998164e-01]
[1.9868465e-05 9.9998009e-01]
[2.6187017e-05 9.9997377e-01]
[1.7995648e-05 9.9998200e-01]
[3.0864394e-04 9.9969137e-01]
[2.2795495e-05 9.9997723e-01]


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


[9.999236e-01 7.637066e-05]
[6.4934284e-05 9.9993503e-01]
[1.9976911e-05 9.9997997e-01]
[3.7957605e-05 9.9996209e-01]
[9.9989939e-01 1.0054953e-04]
[2.6053478e-05 9.9997389e-01]
[9.9989510e-01 1.0485207e-04]
[9.7577693e-05 9.9990237e-01]
[5.9898270e-05 9.9994016e-01]
[1.8199804e-05 9.9998176e-01]
[2.910575e-05 9.999709e-01]
[1.9331931e-04 9.9980670e-01]
[9.99936e-01 6.39796e-05]
[9.9997163e-01 2.8371294e-05]
[5.517639e-05 9.999448e-01]
[2.916006e-05 9.999708e-01]
[2.10250e-05 9.99979e-01]
[2.4904168e-05 9.9997509e-01]
[9.9995708e-01 4.2957683e-05]
[2.4146895e-05 9.9997580e-01]
[2.4339599e-05 9.9997568e-01]
[3.6843532e-05 9.9996316e-01]
[3.8556354e-05 9.9996150e-01]
[2.3857050e-05 9.9997616e-01]
[9.9994290e-01 5.7156958e-05]
[2.1306898e-05 9.9997866e-01]
[2.0084422e-05 9.9997997e-01]
[9.9968696e-01 3.1303099e-04]
[4.286642e-05 9.999571e-01]
[3.2068605e-05 9.9996793e-01]
[2.0625743e-05 9.9997938e-01]
[4.9371133e-04 9.9950624e-01]
[2.9411443e-05 9.9997056e-01]
[9.9992275e-01 7.7227807e-05